In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/productsales/sales.csv


In [ ]:
pip install pyspark

In [4]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("Python Spark") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

22/09/30 04:17:24 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [7]:
from pyspark.sql.window import Window
from pyspark.sql.functions import lag
from pyspark.sql.functions import col
df = spark.read.csv("../input/productsales/sales.csv", header=True, inferSchema=True)

In [6]:
win = Window.partitionBy('id').orderBy('Discount_price')
df1=df.withColumn('perc_price_change', (df.Discount_price - lag(df['Discount_price']).over(win))/100)
df1=df1.withColumn('perc_demand_change',(df.Impression-lag(df['Impression']).over(win))/100)

In [8]:
df1.select("Discount_price","perc_price_change","Impression","perc_demand_change").show()

+--------------+-------------------+----------+------------------+
|Discount_price|  perc_price_change|Impression|perc_demand_change|
+--------------+-------------------+----------+------------------+
|          96.6|               null|         1|              null|
|        106.99|             0.1039|         1|               0.0|
|        174.99| 0.6800000000000002|         1|               0.0|
|        179.99|               0.05|         5|              0.04|
|        184.99|               0.05|         1|             -0.04|
|         189.0|0.04009999999999991|         1|               0.0|
|         199.0|                0.1|         2|              0.01|
|         219.0|                0.2|         1|             -0.01|
|         229.0|                0.1|         1|               0.0|
+--------------+-------------------+----------+------------------+



In [11]:
df2=df1.withColumn("price_elasticiy", df1.perc_demand_change /df1.perc_price_change)
df3=df2.select("Discount_price","perc_price_change","Impression","perc_demand_change",'price_elasticiy')
df3.show()


+--------------+-------------------+----------+------------------+--------------------+
|Discount_price|  perc_price_change|Impression|perc_demand_change|     price_elasticiy|
+--------------+-------------------+----------+------------------+--------------------+
|          96.6|               null|         1|              null|                null|
|        106.99|             0.1039|         1|               0.0|                 0.0|
|        174.99| 0.6800000000000002|         1|               0.0|                 0.0|
|        179.99|               0.05|         5|              0.04|  0.7999999999999999|
|        184.99|               0.05|         1|             -0.04| -0.7999999999999999|
|         189.0|0.04009999999999991|         1|               0.0|                 0.0|
|         199.0|                0.1|         2|              0.01| 0.09999999999999999|
|         219.0|                0.2|         1|             -0.01|-0.04999999999999...|
|         229.0|                